In [1]:
import pandas as pd

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
def restructureDf(df):
    df = df[3:]
    df.columns = df.iloc[0]
    df = df.drop(df.index[0] )
    df = df.reset_index(drop = True)
    df.columns.name = 'index'
    return df

In [4]:
withdrawal_df = pd.read_csv("withdrawal.csv", header = None)
withdrawal_df = restructureDf( withdrawal_df )

In [5]:
deposit_df = pd.read_csv("deposit.csv", header = None)
deposit_df = restructureDf(deposit_df)

In [6]:
gp_df = pd.read_csv("gameplay.csv", header = None, low_memory=False)
gp_df = restructureDf(gp_df)

In [7]:
test = withdrawal_df['User Id'].astype(int)
test.describe()

count    3566.000000
mean      509.116657
std       305.839395
min         2.000000
25%       212.000000
50%       511.500000
75%       795.000000
max       992.000000
Name: User Id, dtype: float64

In [8]:
deposit_df.head()

index,User Id,Datetime,Amount
0,357,01-10-2022 0:03,2000
1,776,01-10-2022 0:03,2500
2,492,01-10-2022 0:06,5000
3,803,01-10-2022 0:07,5000
4,875,01-10-2022 0:09,1500


In [9]:
gp_df.head()
gp_df.rename(columns = {'Games Played' : 'Amount', 'User ID':'User Id'}, inplace = True)
gp_df['User Id'].describe()

count     355266
unique      1000
top          765
freq       24096
Name: User Id, dtype: object

# Merging Dataframes

In [10]:
def groupDf(df):
    df.Datetime = pd.to_datetime(df.Datetime)
    df['slot'] = [0 if i.hour <= 12 else 1 for i in df.Datetime]
    df['day'] = [i.day for i in df.Datetime]
    df['month'] = [i.month for i in df.Datetime]
    df.Amount = df.Amount.astype(int)
    df['User Id'] = df['User Id'].astype(int)
    df = df.groupby(['User Id','month','day','slot']).Amount.agg(['sum'])
    df = df.reset_index(level = [0,1,2,3])
    return df


In [11]:
wd_sum = groupDf(withdrawal_df)
#wd_sum['User Id'].describe()

In [12]:
dp_sum = groupDf(deposit_df)
dp_sum

,User Id,month,day,slot,sum
0,1,10,18,1,5000
1,2,1,10,1,40000
2,2,2,10,0,5000
3,2,6,10,0,40000
4,2,8,10,0,45000
...,...,...,...,...,...
12319,998,10,25,1,1100
12320,998,10,26,0,2000
12321,999,1,10,1,2000
12322,999,10,21,1,2000


In [13]:
gp_sum = groupDf(gp_df)
gp_sum

,User Id,month,day,slot,sum
0,0,4,10,1,1
1,0,5,10,1,1
2,0,6,10,0,1
3,0,10,10,0,1
4,0,10,10,1,1
...,...,...,...,...,...
33991,999,10,31,1,4
33992,999,11,10,0,6
33993,999,11,10,1,3
33994,999,12,10,0,3


In [14]:
gp_sum.rename(columns = {'sum' : 'games'}, inplace = True)
dp_sum.rename(columns = {'sum' : 'deposit'}, inplace = True)
wd_sum.rename(columns = {'sum' : 'withdraw'}, inplace = True)

In [15]:
df = pd.merge(wd_sum, dp_sum, on = ['User Id','day','month','slot'], how = 'outer').merge(gp_sum, on = ['User Id','day','slot','month'], how = 'outer')

In [16]:
df

,User Id,month,day,slot,withdraw,deposit,games
0,2,1,10,1,55000.0,40000.0,1.0
1,2,3,10,0,117349.0,NaN,2.0
2,2,4,10,1,19000.0,NaN,1.0
3,2,5,10,0,36000.0,NaN,1.0
4,2,8,10,1,13000.0,5000.0,1.0
...,...,...,...,...,...,...,...
39004,999,10,31,1,NaN,NaN,4.0
39005,999,11,10,0,NaN,NaN,6.0
39006,999,11,10,1,NaN,NaN,3.0
39007,999,12,10,0,NaN,NaN,3.0


In [17]:
df = df.fillna(0)

## Calculating Score

In [18]:
df['a'] = 0.01 * df.deposit

In [19]:
df['b'] = 0.05 * df.withdraw

In [20]:
df['c'] = 0.001 * (df.deposit - df.withdraw)
df['c'] = [0 if i < 0 else i for i in df.c]

In [21]:
df['d'] = 0.2 * df.games

In [22]:
df['score'] = df.a + df.b + df.c + df.d

In [23]:
df['score'].describe()

count    39009.000000
mean       136.298839
std       1013.141709
min          0.200000
25%          0.400000
50%          2.000000
75%         22.600000
max      55000.000000
Name: score, dtype: float64

## Bonus Allocation ( PART - B )

In [24]:
df_ = df.groupby(['User Id']).score.agg(['sum']).sort_values('sum', ascending= False)

In [25]:
df_ = df_[:50]

In [26]:
df_ = df_ / df_.sum()

In [27]:
df_['reward_value'] = df_ * 50000 

In [28]:
df_

,sum,reward_value
User Id,,
634,0.260986,13049.297431
99,0.043697,2184.863397
162,0.039120,1955.986339
365,0.031045,1552.240634
415,0.030244,1512.221132
920,0.029902,1495.084603
369,0.028472,1423.601263
78,0.025038,1251.924615
587,0.024777,1238.871035


"""
"Part A - Calculating loyalty points

On each day, there are 2 slots for each of which the loyalty points are to be calculated:
S1 from 12am to 12pm 
S2 from 12pm to 12am"				
"Based on the above information and the data provided answer the following questions:
1. Find Playerwise Loyalty points earned by Players in the following slots:-
    a. 2nd October Slot S1
    b. 16th October Slot S2
    b. 18th October Slot S1
    b. 26th October Slot S2
2. Calculate overall loyalty points earned and rank players on the basis of loyalty points in the month of October. 
     In case of tie, number of games played should be taken as the next criteria for ranking.
3. What is the average deposit amount?
4. What is the average deposit amount per user in a month?
5. What is the average number of games played per user?"				
"""

# SOLUTIONS #

In [29]:
df

,User Id,month,day,slot,withdraw,deposit,games,a,b,c,d,score
0,2,1,10,1,55000.0,40000.0,1.0,400.0,2750.00,0.0,0.2,3150.20
1,2,3,10,0,117349.0,0.0,2.0,0.0,5867.45,0.0,0.4,5867.85
2,2,4,10,1,19000.0,0.0,1.0,0.0,950.00,0.0,0.2,950.20
3,2,5,10,0,36000.0,0.0,1.0,0.0,1800.00,0.0,0.2,1800.20
4,2,8,10,1,13000.0,5000.0,1.0,50.0,650.00,0.0,0.2,700.20
...,...,...,...,...,...,...,...,...,...,...,...,...
39004,999,10,31,1,0.0,0.0,4.0,0.0,0.00,0.0,0.8,0.80
39005,999,11,10,0,0.0,0.0,6.0,0.0,0.00,0.0,1.2,1.20
39006,999,11,10,1,0.0,0.0,3.0,0.0,0.00,0.0,0.6,0.60
39007,999,12,10,0,0.0,0.0,3.0,0.0,0.00,0.0,0.6,0.60


## Question number 1

In [30]:
###### 2nd October Slot S1 #######

df[(df.day == 2) & (df.month == 10) & (df.slot == 0)]

#There is no data for this condition

,User Id,month,day,slot,withdraw,deposit,games,a,b,c,d,score


In [31]:
##### 16th October Slot S2

df2 = df[(df.day == 16) & (df.month == 10) & (df.slot == 1)]
df2[['User Id','score']]

 

,User Id,score
81,16,1239.700
196,47,38.558
325,82,3175.000
458,133,400.910
531,157,100.400
...,...,...
38702,989,11.600
38749,990,11.000
38822,992,7.800
38881,996,1.800


In [32]:
####18th October Slot S1

df3 = df[(df.day == 18) & (df.slot == 0) & (df.month == 10)]
df3[['User Id','score']]



,User Id,score
83,16,1955.850
136,31,163.000
170,45,1021.400
198,47,158.539
295,78,802.800
...,...,...
38660,987,8.400
38752,990,15.600
38824,992,9.600
38925,997,0.200


In [33]:
#### 26th October Slot S2

df4 = df[(df.day == 26) & (df.slot == 1) & (df.month == 10)]
df4[['User Id','score']]

,User Id,score
27,7,331.05
34,9,1022.20
94,16,506.75
165,44,25.00
226,50,60.00
...,...,...
38769,990,10.20
38799,991,0.20
38860,995,0.20
38891,996,0.40


## Question Number 02
## Calculate overall loyalty points earned and rank players on the basis of loyalty points in the month of October. In case of tie, number of games played should be taken as the next criteria for ranking.

In [34]:
df5 = df[df.month == 10][['User Id','score','games']]

In [35]:
df5.groupby('User Id')[['score','games']].agg(['sum']).sort_values([('score', 'sum'),('games', 'sum')], ascending = False)

,score,games
,sum,sum
User Id,,
634,586872.00,22.0
365,74769.88,2368.0
369,71231.50,22.0
920,68957.95,614.0
162,64109.06,0.0
...,...,...
49,0.20,1.0
388,0.20,1.0


## Question number 3
### What is the average deposit amount?

In [36]:
plot = df.groupby("User Id").deposit.agg('sum')
plot.mean()

95772.729

## Question number 4
### What is the average deposit amount per user in a month?

In [37]:
plot2 = df.groupby(["month"]).deposit.agg('mean')

In [38]:
plot2.mean()

2396.4734174714263

## Question number 5
### What is the average number of games played per user?

In [39]:
#Average number of games played per user

plot2 = df.groupby("User Id").games.agg('sum')
plot2.mean()

355.267

# PART C

### Loyalty point formule favours the user with heavy deposit amount, from the perspective of company, 
### this factor assumes that user paying more is supposedly generating more revenue when compared to other users, but this alone cannot be considered as metric to measure loyalty, as this measure does not define company's revenue, as Company generate revenue not when the user deposits money but when the user plays.

# Alternative
### Loyalty should be measure by retention and the actual causation variable for profit
### According to me the more useful measure should be, 
### "Average games played per day", "Average Money deposited per Game"
### Another change that's needed to be made in the formula is reducing the weight of deposit amount and removing the factor withdrawal amount from the equation, It points at double calculation, only the difference is enough.